In [1]:
using IntervalArithmetic,IntervalBoxes,IntervalArithmetic.Symbols,Plots,ForwardDiff,LinearAlgebra

In [3]:
begin
    x1 = 1 / 3
    x2 = 2 / 3
    h = 1
    g = 9.8
    f1(y1, y2) = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1)) +
                 sqrt((x1 - x2)^2 + (y1 - y2)^2) /
                 (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2))) +
                 sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))
    fdash1(y1, y2) = ForwardDiff.derivative(y -> f1(y, y2), y1)
    fdash2(y1, y2) = ForwardDiff.derivative(y -> f1(y1, y), y2)
    f0(y) = [fdash1(y[1], y[2]), fdash2(y[1], y[2])]
    X0 = [interval(0.5,0.9), interval(0.5,0.9)]
    J = ForwardDiff.jacobian(f0, X0)
    @show J
    @show det(J)
   
    if in_interval(0,det(J)) 
        P = bisect(X0[1])
        Q = [P[1],X0[2]]
        Q1 = [P[2],X0[2]]
        println(Q)
        println(Q1)
        println(X0)
        @show(ForwardDiff.jacobian(f0, Q))
        @show(ForwardDiff.jacobian(f0, Q1))
    else
        # Xold = Xtemp
        J_inv = inv(J)  # Inverse Jacobian
        mid_X = mid.(X0)
        Z = mid_X.- J_inv* f0(mid_X)
        # Z1 = mid_X - (J_inv[1,1] * f0(mid_X)[1] - J_inv[1,2] * f0(mid_X)[2])
        # println(mid_X)
        # println(f0(mid_X))
        # println(J_inv * f0(mid_X))
        println(mid_X.- J_inv* f0(mid_X))
        # println(Z1)
        println(intersect_interval(X0[1], Z[1]))
        # println(intersect_interval(X0[1], Z1[1]))
        println(intersect_interval(X0[2], Z[2]))
        #return mid_X .- Z* f0(mid_X)
        # Xnew = intersect_interval(Xold, N(Xold, J_inv))
        # Xold = Xnew
    end
    
end


J = Interval{Float64}[[-15.9237, 72.8474]_com_NG [-4.73053, 8.1975]_com_NG; [-4.73053, 8.1975]_com_NG [-3.29725, 14.7397]_com_NG]
det(J) = [-1271.45, 2213.21]_com_NG
Interval{Float64}[[0.5, 0.700001]_com, [0.5, 0.900001]_com]
Interval{Float64}[[0.6999999, 0.900001]_com, [0.5, 0.900001]_com]
Interval{Float64}[[0.5, 0.900001]_com, [0.5, 0.900001]_com]
ForwardDiff.jacobian(f0, Q) = Interval{Float64}[[-2.80248, 5.26685]_com_NG [-2.068, 2.10774]_com_NG; [-2.05263, 2.10774]_com_NG [-1.48921, 7.69611]_com_NG]
ForwardDiff.jacobian(f0, Q1) = Interval{Float64}[[-6.72267, 48.2257]_com_NG [-3.40706, 7.076]_com_NG; [-3.39456, 7.076]_com_NG [-3.27373, 12.311]_com_NG]


2×2 Matrix{Interval{Float64}}:
 [-6.72267, 48.2257]_com_NG  [-3.40706, 7.076]_com_NG
 [-3.39456, 7.076]_com_NG    [-3.27373, 12.311]_com_NG

In [2]:
begin
    x=0.5
    g=9.8
    f(y) = 2*(√(x^2 + (1-y)^2)/√(2*g*(1-y)) + √((1-x)^2 + y^2)/(√(2*g*(1-y)) + √(2*g)));
    fdash(y) = ForwardDiff.derivative(f, y);
    fd(y) =  ForwardDiff.derivative(fdash, y);

    N(X, Y) = mid(X) - fdash(mid(X))*Y;

    function IntervalNewtonExtendedDiv(Xtemp)

        Xold, Xnew, Z = [], [], [];

        if in_interval(0, fd(Xtemp))
            Z = extended_div(interval(1, 1), fd(Xtemp));
            Y1 = intersect_interval(Xtemp, N(Xtemp, Z[1]));
            if !isempty_interval(Y1)
                IntervalNewtonExtendedDiv(Y1);
            end

            Y2 = intersect_interval(Xtemp, N(Xtemp, Z[2]));
            if !isempty_interval(Y2)
                IntervalNewtonExtendedDiv(Y2);
            end

        else
            Xold = Xtemp;
            while diam(Xold) > 0.0001
                Xnew = intersect_interval(Xold, N(Xold, 1/fd(Xold)));
                Xold = Xnew;
            end
            push!(solution, Xnew);
            # return
        end
    end

    solution = [];
    X0 = interval(0, 1);
    IntervalNewtonExtendedDiv(X0);
    @show solution
end

┌ Warning: ill-formed interval [a, b] with a = NaN, b = NaN and decoration d = com. NaI is returned
└ @ IntervalArithmetic C:\Users\SANTHOSHI\.julia\packages\IntervalArithmetic\NcRc8\src\intervals\construction.jl:443
┌ Warning: interval part of NaI
└ @ IntervalArithmetic C:\Users\SANTHOSHI\.julia\packages\IntervalArithmetic\NcRc8\src\intervals\construction.jl:300
┌ Warning: interval part of NaI
└ @ IntervalArithmetic C:\Users\SANTHOSHI\.julia\packages\IntervalArithmetic\NcRc8\src\intervals\construction.jl:300
┌ Warning: invalid interval, NaI is returned
└ @ IntervalArithmetic C:\Users\SANTHOSHI\.julia\packages\IntervalArithmetic\NcRc8\src\intervals\construction.jl:462
┌ Warning: interval part of NaI
└ @ IntervalArithmetic C:\Users\SANTHOSHI\.julia\packages\IntervalArithmetic\NcRc8\src\intervals\construction.jl:300
┌ Warning: invalid interval, NaI is returned
└ @ IntervalArithmetic C:\Users\SANTHOSHI\.julia\packages\IntervalArithmetic\NcRc8\src\intervals\construction.jl:462
┌ Warning: i

solution = Any[Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], [0.999755, 0.999783]_trv_NG, [0.999511, 0.999565]_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, ∅_trv_NG, [0.206164, 0.206165]_trv_NG]


56-element Vector{Any}:
         Any[]
         Any[]
         Any[]
         Any[]
         Any[]
         Any[]
         Any[]
         Any[]
         Any[]
         Any[]
        ⋮
 ∅_trv_NG
 ∅_trv_NG
 ∅_trv_NG
 ∅_trv_NG
 ∅_trv_NG
 ∅_trv_NG
 ∅_trv_NG
 ∅_trv_NG
       [0.206164, 0.206165]_trv_NG

In [ ]:
begin
    x1 = 1 / 3
    x2 = 2 / 3
    h = 1
    g = 9.8

    # Function definition
    f1(y1, y2) = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1)) +
                 sqrt((x1 - x2)^2 + (y1 - y2)^2) /(sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2))) +
                 sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))

    fdash1(y1, y2) = ForwardDiff.derivative(y -> f1(y, y2), y1)
    fdash2(y1, y2) = ForwardDiff.derivative(y -> f1(y1, y), y2)

    f0(y) = [fdash1(y[1], y[2]), fdash2(y[1], y[2])]
    function N(X, Z)
        mid_X = mid.(X)
        println(mid_X - Z* f0(mid_X))
        return mid_X - Z* f0(mid_X)
    end
    # Recursive Interval Newton function
    function IntervalNewtonExtendedDiv(Xtemp, solution)
        J = ForwardDiff.jacobian(f0, Xtemp)
        P = det(J)
        #println(Xtemp)
        # println(J)
        # println(P)
        if in_interval(0, P)
            # Perform extended division
            if diam(Xtemp[1]) >= diam(Xtemp[2]) 
                if diam(Xtemp[1]) > 0.0001
                    Y = bisect(Xtemp[1])
                    Q = [Y[1],Xtemp[2]]
                    Q1 = [Y[2],Xtemp[2]]
                    # println(Q)
                    # println(Q1)
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution) 
                end
            else
                if diam(Xtemp[2]) > 0.0001
                    Y1 = bisect(Xtemp[2])
                    Q = [Xtemp[1],Y1[1]]
                    Q1 = [Xtemp[1],Y1[2]]
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution)
                end
            end
        else
            # println(Xtemp)
            Xold = Xtemp
            while norm(f0(mid.(Xold))) > 1e-4
                # println(Xold)
                J = ForwardDiff.jacobian(f0, Xold)
                println(J)
                J_inv = inv(J)
                Z1 =  N(Xold, J_inv)
                Xnew1 = intersect_interval(Xold[1],Z1[1])
                Xnew2 = intersect_interval(Xold[2],Z1[2])
                println(Xnew1)
                println(Xnew2)
                Xold =[Xnew1,Xnew2]
            end
            println(Xold)
            push!(solution, Xold)
            return
        end
    end

    solution = []
    X0 = [interval(0.1, 0.9), interval(0.1, 0.9)]
    IntervalNewtonExtendedDiv(X0, solution)
    solution
end

Interval{Float64}[[0.315409, 0.417721]_com_NG [-0.363347, -0.342276]_com_NG; [-0.363347, -0.342276]_com_NG [0.673318, 0.728275]_com_NG]
Interval{Float64}[[-0.427673, 1.23612]_com_NG, [-0.65762, 1.00617]_com_NG]
[0.0999999, 0.125]_trv_NG
[0.0999999, 0.125]_trv_NG
Interval{Float64}[[0.315409, 0.417721]_trv_NG [-0.363347, -0.342276]_trv_NG; [-0.363347, -0.342276]_trv_NG [0.673318, 0.728275]_trv_NG]
Interval{Float64}[[-0.427673, 1.23612]_com_NG, [-0.65762, 1.00617]_com_NG]
[0.0999999, 0.125]_trv_NG
[0.0999999, 0.125]_trv_NG
Interval{Float64}[[0.315409, 0.417721]_trv_NG [-0.363347, -0.342276]_trv_NG; [-0.363347, -0.342276]_trv_NG [0.673318, 0.728275]_trv_NG]
Interval{Float64}[[-0.427673, 1.23612]_com_NG, [-0.65762, 1.00617]_com_NG]
[0.0999999, 0.125]_trv_NG
[0.0999999, 0.125]_trv_NG
Interval{Float64}[[0.315409, 0.417721]_trv_NG [-0.363347, -0.342276]_trv_NG; [-0.363347, -0.342276]_trv_NG [0.673318, 0.728275]_trv_NG]
Interval{Float64}[[-0.427673, 1.23612]_com_NG, [-0.65762, 1.00617]_com_NG]


In [ ]:
begin
    x1 = 1 / 3
    x2 = 2 / 3
    h = 1
    g = 9.8

    # Function definition
    f1(y1, y2) = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1)) +
                 sqrt((x1 - x2)^2 + (y1 - y2)^2) /(sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2))) +
                 sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))

    fdash1(y1, y2) = ForwardDiff.derivative(y -> f1(y, y2), y1)
    fdash2(y1, y2) = ForwardDiff.derivative(y -> f1(y1, y), y2)

    f0(y) = [fdash1(y[1], y[2]), fdash2(y[1], y[2])]
    # function N(X, Z)
    #     mid_X = mid.(X)
    #     #println(mid_X - Z* f0(mid_X))
    #     return mid_X - Z* f0(mid_X)
    # end

    function N(X, Z)
        mid_X = mid.(X)  # Compute midpoints of the intervals
        f_val = f0(mid_X)  # Evaluate f0 at the midpoints

        # Perform interval arithmetic for the matrix-vector multiplication
        result1 = mid_X[1] - (Z[1, 1] * f_val[1]) - (Z[1, 2] * f_val[2])
        result2 = mid_X[2] - (Z[2, 1] * f_val[1]) - (Z[2, 2] * f_val[2])

        return [result1, result2]
    end
    # Recursive Interval Newton function
    function IntervalNewtonExtendedDiv(Xtemp, solution)
        J = ForwardDiff.jacobian(f0, Xtemp)
        P = det(J)
        println(Xtemp)
        # println(J)
        # println(P)
        if in_interval(0, P)
            # Perform extended division
            if diam(Xtemp[1]) >= diam(Xtemp[2]) 
                if diam(Xtemp[1]) > 0.001
                    Y = bisect(Xtemp[1])
                    Q = [Y[1],Xtemp[2]]
                    Q1 = [Y[2],Xtemp[2]]
                    # println(Q)
                    # println(Q1)
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution) 
                end
            else
                if diam(Xtemp[2]) > 0.001
                    Y1 = bisect(Xtemp[2])
                    Q = [Xtemp[1],Y1[1]]
                    Q1 = [Xtemp[1],Y1[2]]
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution)
                end
            end
        else
            # println(Xtemp)
            Xold = Xtemp
            for i in 1 : 3
                J = ForwardDiff.jacobian(f0, Xold)
                #println(det(J))
                J_inv = inv(J)
                Z1 =  N(Xold, J_inv)
                Xnew1 = intersect_interval(Xold[1],Z1[1])
                Xnew2 = intersect_interval(Xold[2],Z1[2])
                # println(Xnew1)
                # println(Xnew2)
                Xold =[Xnew1,Xnew2]
                println(Xold)
            end
            println(Xold)
            push!(solution, Xold)
            return
        end
    end

    solution = []         
    #X0 = [interval(0.3586657815936687, 0.3589657815936687), interval(0.13651311744833685, 0.13671311744833685)]
    X0 = [interval(0.0,1.0), interval(0.0,1.0)]
    IntervalNewtonExtendedDiv(X0, solution)
    solution
end


In [ ]:
begin
    x1 = 1 / 4
    x2 = 2 / 4
    x3 = 3 / 4
    h = 1
    g = 9.8
    f1(y1, y2, y3) = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1)) +
             sqrt((x1 - x2)^2 + (y1 - y2)^2) / (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2))) +
             sqrt((y2 - y3)^2 + (x2 - x3)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * (h - y3))) +
             sqrt(y3^2 + (x3 - h)^2) / (sqrt(2 * g * (h - y3)) + sqrt(2 * g * h))

# Derivative functions
    fdash1(y1, y2, y3) = ForwardDiff.derivative(y -> f1(y, y2, y3), y1)
    fdash2(y1, y2, y3) = ForwardDiff.derivative(y -> f1(y1, y, y3), y2)
    fdash3(y1, y2, y3) = ForwardDiff.derivative(y -> f1(y1, y2, y), y3)

    f0(y) = [fdash1(y[1], y[2], y[3]), fdash2(y[1], y[2], y[3]), fdash3(y[1], y[2], y[3])]
    function N(X, Z)
        mid_X = mid.(X)
        println(mid_X .- Z* f0(mid_X))
        return mid_X .- Z* f0(mid_X)
    end
    # Recursive Interval Newton function
    function IntervalNewtonExtendedDiv(Xtemp, solution)
        J = ForwardDiff.jacobian(f0, Xtemp)
        P = det(J)
        #println(Xtemp)
        # println(J)
        # println(P)
        if in_interval(0, P)
            # Perform extended division
            if diam(Xtemp[1]) >= diam(Xtemp[2]) && diam(Xtemp[1]) >= diam(Xtemp[3])
                if diam(Xtemp[1]) > 0.0001
                    Y = bisect(Xtemp[1])
                    Q = [Y[1],Xtemp[2],Xtemp[3]]
                    Q1 = [Y[2],Xtemp[2],Xtemp[3]]
                    # println(Q)
                    # println(Q1)
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution) 
                end
            elseif diam(Xtemp[2]) >= diam(Xtemp[3])
                if diam(Xtemp[2]) > 0.0001
                    Y1 = bisect(Xtemp[2])
                    Q = [Xtemp[1],Y1[1],Xtemp[3]]
                    Q1 = [Xtemp[1],Y1[2],Xtemp[3]]
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution)
                end
            else
                if diam(Xtemp[3]) > 0.0001
                    Y2 = bisect(Xtemp[3])
                    Q = [Xtemp[1],Xtemp[2],Y2[1]]
                    Q1 = [Xtemp[1],Xtemp[2],Y2[2]]
                    IntervalNewtonExtendedDiv(Q,solution)
                    IntervalNewtonExtendedDiv(Q1,solution)
                    
                end
            end
        else
            # println(Xtemp)
            Xold = Xtemp
            while norm(f0(mid.(Xold))) > 1e-4
                # println(Xold)
                J = ForwardDiff.jacobian(f0, Xold)
                println(J)
                J_inv = inv(J)
                Z1 =  N(Xold, J_inv)
                Xnew1 = intersect_interval(Xold[1],Z1[1])
                Xnew2 = intersect_interval(Xold[2],Z1[2])
                Xnew3 = intersect_interval(Xold[3],Z1[3])
                println(Xnew1)
                println(Xnew2)
                Xold =[Xnew1,Xnew2,Xnew3]
            end
            push!(solution, Xold)
            return
        end
    end

    solution = []
    X0 = [interval(0, 1), interval(0, 1), interval(0, 1)]
    IntervalNewtonExtendedDiv(X0, solution)
    solution

end

In [ ]:
using IntervalArithmetic
using ForwardDiff
using LinearAlgebra

# Constants
const x1 = 1 / 3
const x2 = 2 / 3
const h = 1.0
const g = 9.8

# Define the objective function
function f1(y1, y2)
    # Avoid singularities at h - y1 = 0 or h - y2 = 0
    term1 = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1))
    term2 = sqrt((x1 - x2)^2 + (y1 - y2)^2) / (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2)))
    term3 = sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))

    return term1 + term2 + term3
end

# Partial derivatives
fdash1(y1, y2) = ForwardDiff.derivative(y -> f1(y, y2), y1)
fdash2(y1, y2) = ForwardDiff.derivative(y -> f1(y1, y), y2)

# Gradient vector
f0(y) = [fdash1(y[1], y[2]), fdash2(y[1], y[2])]

# Newton iteration step
function N(X, Z)
    mid_X = mid.(X)
    return mid_X - Z * f0(mid_X)
end

# Bisect an interval into two halves
function bisect(I::Interval)
    m = mid(I)
    return Interval(inf(I), m), Interval(m, sup(I))
end

# Interval Newton method with extended division
function IntervalNewtonExtendedDiv(Xtemp, solution)
    # Evaluate Jacobian at midpoint
    J = ForwardDiff.jacobian(f0, mid.(Xtemp))
    detJ = det(J)

    # If Jacobian is near singular, subdivide
    if 0 ∈ detJ
        if diam(Xtemp[1]) >= diam(Xtemp[2])
            if diam(Xtemp[1]) > 1e-4
                I1, I2 = bisect(Xtemp[1])
                IntervalNewtonExtendedDiv([I1, Xtemp[2]], solution)
                IntervalNewtonExtendedDiv([I2, Xtemp[2]], solution)
            end
        else
            if diam(Xtemp[2]) > 1e-4
                I1, I2 = bisect(Xtemp[2])
                IntervalNewtonExtendedDiv([Xtemp[1], I1], solution)
                IntervalNewtonExtendedDiv([Xtemp[1], I2], solution)
            end
        end
    else
        # Perform Interval Newton iteration
        Xold = Xtemp
        while norm(f0(mid.(Xold))) > 1e-4
            J = ForwardDiff.jacobian(f0, mid.(Xold))
            J_inv = inv(J)
            Z = N(Xold, J_inv)

            Xnew1 = intersect(Xold[1], Z[1])
            Xnew2 = intersect(Xold[2], Z[2])

            if isempty(Xnew1) || isempty(Xnew2)
                return  # No solution in this interval
            end

            Xold = [Xnew1, Xnew2]
        end

        # Store refined interval containing solution
        push!(solution, Xold)
    end
end

# Run the algorithm over the domain [0, 1] × [0, 1]
solution = []
X0 = [interval(0.0, 1.0), interval(0.0, 1.0)]
IntervalNewtonExtendedDiv(X0, solution)

println("\nSolutions found:")
for sol in solution
    println("y1 ∈ $(sol[1]), y2 ∈ $(sol[2])")
end


In [ ]:
x1 = 1 / 3
x2 = 2 / 3
h = 1
g = 9.8

f1(y1, y2) = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1)) +
                 sqrt((x1 - x2)^2 + (y1 - y2)^2) /(sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2))) +
                 sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))

fdash1(y1, y2) = ForwardDiff.derivative(y -> f1(y, y2), y1)
fdash2(y1, y2) = ForwardDiff.derivative(y -> f1(y1, y), y2)

f0(y) = [fdash1(y[1], y[2]), fdash2(y[1], y[2])]
y = [interval(0.015625, 0.0166016), interval(0.496093, 0.498047)]
println(f1(y[1],y[2]))
println(norm(f0(mid.(y))))


f(y) = sqrt(y^2 + 1)
fdash(y) = ForwardDiff.derivative(f)
println(fdash((0.1,1.0)))



In [ ]:
using IntervalArithmetic
using IntervalRootFinding
using ForwardDiff

# Constants
x1 = 1 / 3
x2 = 2 / 3
h = 1
g = 9.8

# Function definition
f1(y1, y2) = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1)) +
             sqrt((x1 - x2)^2 + (y1 - y2)^2) / (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2))) +
             sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))

# Derivatives using ForwardDiff
fdash1(y1, y2) = ForwardDiff.derivative(y -> f1(y, y2), y1)
fdash2(y1, y2) = ForwardDiff.derivative(y -> f1(y1, y), y2)

# Function to find roots
f0(y) = [fdash1(y[1], y[2]), fdash2(y[1], y[2])]

# Define the intervals for y1 and y2
y_intervals = [interval(0.0, 1.0), interval(0.0, 1.0)]

# Find the roots using the Interval Newton method
found_roots = roots(f0, y_intervals)  # Use a different variable name to avoid conflict

# Display the results
println("Roots found: ", found_roots)